In [ ]:
# !pip install tensorflow tqdm tensorlayerx numpy

In [ ]:
import tensorflow;
import os;
import tensorlayerx;
from tensorlayerx import convert_to_tensor;

import numpy;

from tensorflow.keras.datasets import mnist;

from tensorlayerx.nn import Sequential, Conv2d, MaxPool2d, Flatten, Linear, Module, Input;
from tensorlayerx.model import Model;
from tensorlayerx.optimizers import Adam;
from tensorlayerx.losses import softmax_cross_entropy_with_logits;
from tensorlayerx.metrics import acc;
from tensorlayerx.dataflow import IterableDataset;

os.environ["TL_BACKEND"] = "tensorflow";

In [ ]:
print(tensorflow.config.list_physical_devices('GPU'));


import tensorflow as tf

# Set memory growth for GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # Set memory growth
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("Memory growth set for GPU")
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Memory growth set for GPU


In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data();

train_images = train_images;
train_labels= train_labels;

test_images = test_images;
test_labels= test_labels;

# Split the test and val by 50:50
test_val_images_split = numpy.array_split(test_images, 2);
test_val_labels_split = numpy.array_split(test_labels, 2);

test_images = test_val_images_split[0];
test_labels = test_val_labels_split[0];

val_images = test_val_images_split[1];
val_labels = test_val_labels_split[1];

In [ ]:
# Recall step 1
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255;
test_images = test_images.reshape((5000, 28, 28, 1)).astype('float32') / 255;
val_images = val_images.reshape((5000, 28, 28, 1)).astype('float32') / 255;

In [ ]:
train_images.shape, test_images.shape, val_images.shape, train_labels.shape, test_labels.shape, val_labels.shape

((60000, 28, 28, 1),
 (5000, 28, 28, 1),
 (5000, 28, 28, 1),
 (60000,),
 (5000,),
 (5000,))

In [ ]:
tensorflow.experimental.numpy.experimental_enable_numpy_behavior()

class DatasetLoader(IterableDataset):
    def __init__(self, feature, label, batch_size = 64):
        self.data = convert_to_tensor(feature)
        self.label = label
        self.batch_size = batch_size

    def __getitem__(self, index):
        data = self.data[index];
        label = self.label[index];

        return data, label;

    def __len__(self):
        return len(self.data);

    def __iter__(self):
        for i in range(0, len(self.data), self.batch_size):
            batch_data = self.data[i:i+self.batch_size]
            batch_label = self.label[i:i+self.batch_size]

            yield batch_data, batch_label.astype("int32")

In [ ]:
class CNNModel (Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.input = Input(shape = (8, 28, 28, 1));

        self.conv1 = Conv2d(out_channels = 64, kernel_size = (3, 3), act = "relu", name = "conv1");
        self.pool1 = MaxPool2d(kernel_size = (2, 2), padding = "SAME", name = "pool1");

        self.conv2 = Conv2d(out_channels = 64, kernel_size = (3, 3), act = "relu", name = "conv2");
        self.pool2 = MaxPool2d(kernel_size = (2, 2), padding = "SAME", name = "pool2");

        self.flat = Flatten(name = "flat");

        self.dense = Linear(out_features = 64, in_features = 3136, act = "relu", name = "dense");

        self.output = Linear(out_features = 10, name = "output");

    def forward(self, x):
        x = self.input;
        x = self.conv1(x);
        x = self.pool1(x);

        x = self.conv2(x);
        x = self.pool2(x);

        x = self.flat(x);
        x = self.dense(x);
        x = self.output(x);

        return x;

    def construct(self, x):
        x = self.input;
        x = self.conv1(x);
        x = self.pool1(x);

        x = self.conv2(x);
        x = self.pool2(x);

        x = self.flat(x);
        x = self.dense(x);
        x = self.output(x);

        return x;


    def compile(self):
        model = Sequential([
            self.input,
            self.conv1,
            self.pool1,
            self.conv2,
            self.pool2,
            self.flat,
            self.dense,
            self.output
        ]);

        print("------------------------------------------------")
        print("               MNIST Tensorlayerx               ");
        print("------------------------------------------------")
        print("Neuron: ", sum(layer.out_features for layer in model if hasattr(layer, 'out_features')))
        print(model);

        return model;

network = CNNModel();

loss_fn = softmax_cross_entropy_with_logits;
metric_function = acc;
optimizer = Adam(lr = 1e-3);
model = Model(network, loss_fn = loss_fn, optimizer = optimizer, metric_function = metric_function);


INFO:tensorlayerx:Input  _inputlayer_6: (8, 28, 28, 1)
INFO:tensorlayerx:Conv2d conv1: out_channels : 64 kernel_size: (3, 3) stride: (1, 1) pad: SAME act: ReLU
INFO:tensorlayerx:MaxPool2d pool1: kernel_size: (2, 2) stride: (2, 2) padding: SAME return_mask: False
INFO:tensorlayerx:Conv2d conv2: out_channels : 64 kernel_size: (3, 3) stride: (1, 1) pad: SAME act: ReLU
INFO:tensorlayerx:MaxPool2d pool2: kernel_size: (2, 2) stride: (2, 2) padding: SAME return_mask: False
INFO:tensorlayerx:Flatten flat:
INFO:tensorlayerx:Linear  dense: 64 ReLU
INFO:tensorlayerx:Linear  output: 10 No Activation


In [ ]:
train_dataset = DatasetLoader(train_images, train_labels, 8);
val_dataset = DatasetLoader(val_images, val_labels, 8);
test_dataset = DatasetLoader(test_images, test_labels, 8);

model.train(n_epoch = 25, train_dataset = train_dataset, test_dataset = val_dataset, print_freq = 1);

Output()

Epoch 1 of 25 took 77.40816712379456

train loss: 2.3015966415405273

train acc:  0.11225

val loss: 2.3010640144348145

...

Epoch 25 of 25 took 76.57949686050415

train loss: 2.301466941833496

train acc:  0.11236666666666667

val loss: 2.3010616302490234

val acc:  0.1128

In [ ]:
model.eval(test_dataset = test_dataset);